In [46]:

import random
import numpy as np
import pandas as pd
from itertools import combinations
from tqdm.notebook import tqdm

In [3]:
def create_random_strand(len_strand):
    choices = ['A', 'C', 'T', 'G']
    return "".join([random.choice(choices) for i in range(len_strand)])


In [37]:

# First let us see if squigulator lets us seperate out into cycle reads, cause that will make life much simpler

n_motifs_per_read = 32
n_unique_motifs = 8
n_spacer_motifs = 8
motifs_per_payload = 4
len_motif = 20
len_link = 10
n_cycles = 8

motif_choices = [create_random_strand(len_motif) for i in range(n_unique_motifs)]
link_choices = [create_random_strand(len_link) for i in range(n_spacer_motifs)]
motif_indices = [1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
n_reads = 20000
reads_base_level = []
reads_motif_level = []
motif_picks = list(combinations(motif_indices, 4)) # Storing all possible combinations to draw from

for read in tqdm(range(n_reads)):
    read_base_level = ""
    read_motif_level = []
    for cycle_position in range(n_cycles):
        payload_motifs = random.choice(motif_picks)
        payload_read_base_level = "".join([f"{link_choices[cycle_position]}{motif_choices[i-1]}{link_choices[cycle_position]}" for i in payload_motifs])
        read_base_level += payload_read_base_level
        read_motif_level.append(payload_motifs)
    reads_base_level.append(read_base_level)
    reads_motif_level.append(read_motif_level)

  0%|          | 0/20000 [00:00<?, ?it/s]

In [63]:
def create_spacer_sequence(cycles):

    spacer_sequence = []

    cycle_number = 9

    """
    if cycles.isinstance(str):
        payload = eval(cycles)
    """
        
    for i in cycles:
        for j in i:
            spacer_sequence.append(cycle_number)
            spacer_sequence.append(j)
            spacer_sequence.append(cycle_number)
        cycle_number += 1

    return spacer_sequence

In [64]:
create_spacer_sequence(reads_motif_level[0])

[9,
 1,
 9,
 9,
 5,
 9,
 9,
 6,
 9,
 9,
 8,
 9,
 10,
 4,
 10,
 10,
 6,
 10,
 10,
 7,
 10,
 10,
 8,
 10,
 11,
 2,
 11,
 11,
 3,
 11,
 11,
 5,
 11,
 11,
 8,
 11,
 12,
 3,
 12,
 12,
 4,
 12,
 12,
 5,
 12,
 12,
 6,
 12,
 13,
 2,
 13,
 13,
 3,
 13,
 13,
 4,
 13,
 13,
 6,
 13,
 14,
 1,
 14,
 14,
 2,
 14,
 14,
 4,
 14,
 14,
 6,
 14,
 15,
 3,
 15,
 15,
 5,
 15,
 15,
 6,
 15,
 15,
 7,
 15,
 16,
 2,
 16,
 16,
 3,
 16,
 16,
 6,
 16,
 16,
 8,
 16]

In [54]:
reads_motif_level[0][0]

[[...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
